In [ ]:
import os
import numpy as np
import pandas as pd
from numpy import log10
import bioinfokit.analys

## SQuIRE result

In [ ]:
def CPM_norm(d):

    nm = bioinfokit.analys.norm()
    nm.cpm(df=d)
    
    cpm_ = nm.cpm_norm
    
    return cpm_


def order(df):

    df = df[['A_FST','B_FST','A_SND','B_SND','A_TRD','B_TRD',
             'C_FST','D_FST','C_SND','D_SND','C_TRD','D_TRD',
             'E_FST','F_FST','E_SND','F_SND','E_TRD','F_TRD',
             'G_FST','H_FST','G_SND','H_SND','G_TRD','H_TRD']]

    return df


def interpolation(df):

    df = df[['A_FST','B_FST','A_SND','B_SND','A_TRD','B_TRD',
             'C_FST','D_FST','C_SND','D_SND','C_TRD','D_TRD',
             'E_FST','F_FST','E_SND','F_SND','E_TRD','F_TRD',
             'G_FST','H_FST','G_SND','H_SND','G_TRD','H_TRD']]
    
    df.columns = [1,1,2,2,3,3,4,4,5,5,6,6,7,7,8,8,9,9,10,10,11,11,12,12]

    df_T = df.transpose()
    df_1 = df_T.iloc[0:6,:]
    df_2 = df_T.iloc[6:12,:]
    df_3 = df_T.iloc[12:18,:]
    df_4 = df_T.iloc[18:24,:]

    for i in [df_1,df_2,df_3,df_4]:
        null = pd.DataFrame(i.isnull().sum())
        null.columns = ['count']
        null_N = null[null['count']==1]
        for j in null_N.index:
            i[j] = i[j].interpolate(method = 'values',limit=1,limit_direction='both')
    
    df_filled = pd.concat([df_1,df_2,df_3,df_4])
    df_filled = df_filled.transpose()
    
    df_filled.columns = ['A_FST','B_FST','A_SND','B_SND','A_TRD','B_TRD',
                         'C_FST','D_FST','C_SND','D_SND','C_TRD','D_TRD',
                         'E_FST','F_FST','E_SND','F_SND','E_TRD','F_TRD',
                         'G_FST','H_FST','G_SND','H_SND','G_TRD','H_TRD']
    

    return df_filled


def merging(macaca):

    path_dir = './source_data/squire_count_%s' % macaca

    file_list = os.listdir(path_dir)

    GeneCount = pd.DataFrame(columns = ['gene_chr'])
    GeneCount = GeneCount.set_index('gene_chr')

    GeneCPM = pd.DataFrame(columns = ['gene_chr'])
    GeneCPM = GeneCPM.set_index('gene_chr')

    TECount =pd.DataFrame(columns = ['TE_ID'])
    TECount = TECount.set_index('TE_ID')

    TECPM =pd.DataFrame(columns = ['TE_ID'])
    TECPM = TECPM.set_index('TE_ID')

    GeneTPM =pd.DataFrame(columns = ['gene_chr'])
    GeneTPM = GeneTPM.set_index('gene_chr')


    for i in file_list:

        ###GeneCount
        if i.endswith('_refGenecounts.txt'):
            file = './source_data/squire_count_%s/%s' % (macaca,i)
            x = pd.read_csv(file,
                            sep = '\t',
                            names=["chr","start","end","gene","fpkm","strand","count","Accession"])
        
            x = x[x['chr'].isin(['chr1','chr2','chr3','chr4','chr5','chr6','chr7','chr8','chr9','chr10',
                                'chr11','chr12','chr13','chr14','chr15','chr16','chr17','chr18','chr19','chr20','chrX','chrMT'])]
            x['start_str'] = x['start'].astype(str)
            x['gene_chr'] = x['gene']+'_'+x['chr']+'_'+x['start_str']
            x = x[['gene_chr','count']]
            x = x.set_index('gene_chr')
        
            ID = i.split('_ref')[0]

            if macaca == 'mfa':
                ID = ID.split('_')[1]+'_'+ID.split('_')[0]
            elif macaca == 'mml':
                pass
            
            x.columns = [ID]

            ###CPM_conversion
            x_cpm = CPM_norm(x)

            GeneCount = pd.concat([GeneCount,x],join='outer',axis=1)
            GeneCPM = pd.concat([GeneCPM,x_cpm],join='outer',axis=1)


        ###TECount
        if macaca == 'mfa' and i.endswith('_TEcounts.txt'):
            file = './source_data/squire_count_%s/%s' % (macaca,i)        
            y = pd.read_csv(file,
                            sep = '\t',
                            header = 0)
            y = y[y['tx_chr'].isin(['chr1','chr2','chr3','chr4','chr5','chr6','chr7','chr8','chr9','chr10',
                                   'chr11','chr12','chr13','chr14','chr15','chr16','chr17','chr18','chr19','chr20','chrX','chrMT'])]
            y = y[['TE_ID','tot_counts']]
            y = y.groupby('TE_ID').sum()
            ID = i.split('_TE')[0]

            if macaca == 'mfa':
                ID = ID.split('_')[1]+'_'+ID.split('_')[0]
            elif macaca == 'mml':
                pass
            
            y.columns = [ID]

            ###CPM_conversion
            TECount = pd.concat([TECount,y],join='outer',axis=1)
            
    
        ###GeneTPM
        if i.endswith('_abund.txt'):
            file = './source_data/squire_count_%s/%s' % (macaca,i)
            z = pd.read_csv(file,
                            sep = '\t',
                            header = 0)
        
            z = z[z['Reference'].isin(['chr1','chr2','chr3','chr4','chr5','chr6','chr7','chr8','chr9','chr10',
                                   'chr11','chr12','chr13','chr14','chr15','chr16','chr17','chr18','chr19','chr20','chrX','chrMT'])]

            z['start_str'] = z['Start'].astype(str)
            z['gene_chr'] = z['Gene ID']+'_'+z['Reference']+'_'+z['start_str']
            z = z[['gene_chr','TPM']]
            z = z.set_index('gene_chr')
            ID = i.split('_abund')[0]
            
            if macaca == 'mfa':
                ID = ID.split('_')[1]+'_'+ID.split('_')[0]
            elif macaca == 'mml':
                pass
            
            z.columns = [ID]
        
            GeneTPM = pd.concat([GeneTPM,z],join='outer',axis=1)
    
    if macaca == 'mfa':
        return GeneCount, GeneCPM, TECount, GeneTPM
    if macaca == 'mml':
        return GeneCount, GeneCPM, GeneTPM

In [ ]:
def parsing(macaca):

    ### merge
    if macaca == 'mfa':
        GeneCount, GeneCPM, TECount, GeneTPM = merging(macaca)
    if macaca == 'mml':
        GeneCount, GeneCPM, GeneTPM = merging(macaca)

    ### order
    if macaca == 'mfa':
        GeneCount = order(GeneCount)
        GeneCPM = order(GeneCPM)
        TECount = order(TECount)
        GeneTPM = order(GeneTPM)

    ### GeneCount drop NAN
    GeneCount_dropna = GeneCount.dropna()

    ### GeneCPM drop NAN
    GeneCPM_dropna = GeneCPM.dropna()

    ### GeneTPM drop NAN
    GeneTPM_dropna = GeneTPM.dropna()
    
    if macaca == 'mfa':
        ### TECount drop NAN
        TECount_in = interpolation(TECount)
        TECount_in_dropna = TECount_in.dropna()       
        TECount_in_dropna.to_csv('./source_data/count_result/TECount_filled_%s.txt'%(macaca),sep='\t',index=True,header=True)
    
    GeneCount_dropna.to_csv('./source_data/count_result/GeneCount_%s.txt'%(macaca),sep='\t',index=True,header=True)
    GeneCPM_dropna.to_csv('./source_data/count_result/GeneCPM_%s.txt'%(macaca),sep='\t',index=True,header=True)
    GeneTPM_dropna.to_csv('./source_data/count_result/GeneTPM_TIMER_sample_%s.txt'%(macaca),sep='\t',index=True,header=True)

In [ ]:
#### Macaca fascicularis
parsing('mfa')

#### Macaca mulatta
#parsing('mml')

## Kallisto results

In [ ]:
file = './source_data/kallisto/Trpt_kallisto_readcount.txt'
kalisto_readcount = pd.read_csv(file, sep = '\t', header = 0)

#print(kalisto_readcount)

TrptCPM = pd.DataFrame(columns = ['Transcript'])
TrptCPM = TrptCPM.set_index('Transcript')


for i in kalisto_readcount.columns:
    A = kalisto_readcount[[i]]
    A = CPM_norm(A)
    A = A.reset_index()
    A.columns = ['Transcript',i]
    A = A.set_index('Transcript')
    TrptCPM = pd.concat([TrptCPM,A],join='outer',axis=1)
    
TrptCPM_dropna = TrptCPM.dropna()
TrptCPM_dropna.to_csv('./source_data/kallisto/TrptCPM_kallisto_mfa.txt',sep='\t',index=True,header=True)